# Facial Recognition and Verification Project

### Importing Dependencies

In [5]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Functional API
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

### Creating Folder Structures

In [7]:
ANC_PATH = os.path.join('data', 'anchor')
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')

In [9]:
os.makedirs(ANC_PATH)
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)

In [10]:
!tar -xf lfw.tgz

In [19]:
for directory in os.listdir('lfw'):
    dir_path = os.path.join('lfw', directory)
    if os.path.isdir(dir_path):
        for file in os.listdir(dir_path):
            EX_PATH = os.path.join(dir_path, file)
            NEW_PATH = os.path.join(NEG_PATH, file)
            os.replace(EX_PATH, NEW_PATH)

### Opening Webcam

In [34]:
import uuid

In [36]:
img_name = os.path.join(ANC_PATH, f'{uuid.uuid1()}.jpg')
print(img_name)

data\anchor\6c63a0b1-89ac-11ee-a1b9-48684ad16b08.jpg


In [39]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)

    frame = cv2.resize(frame, (250, 250))
    # print(frame.shape)
    # frame = frame[120:120+250, 200:200+250, :]

    if cv2.waitKey(1) & 0XFF == ord('a'):
        img_name = os.path.join(ANC_PATH, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(img_name, frame)
    if cv2.waitKey(1) & 0XFF == ord('p'):
        img_name = os.path.join(POS_PATH, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(img_name, frame)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()

### Getting Image Directories

In [40]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(600)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(600)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(600)

In [42]:
dir_test = anchor.as_numpy_iterator()

In [43]:
dir_test.next()

b'data\\anchor\\32c9640f-89ad-11ee-bfdf-48684ad16b08.jpg'

### Preprocessing

In [48]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100, 100))
    img = img/255.0

    return img

### Creating Labelled Dataset

In [44]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

### Build Train and Test Partition

In [45]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

### Dataloader Pipeline

In [49]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

#### Training Partition

In [59]:
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

#### Test Partition

In [69]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

### Embedding Layer

In [70]:
def make_embedding():
    inp = Input(shape=(100, 100, 3), name='input_image')
    
    c1 = Conv2D(64, (10, 10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2, 2), padding='same')(c1)

    c2 = Conv2D(128, (7, 7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2, 2), padding='same')(c2)

    c3 = Conv2D(128, (4, 4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2, 2), padding='same')(c3)

    c4 = Conv2D(256, (4, 4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)


    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [79]:
embedding = make_embedding()

In [ ]:
embedding.summary()

### Distance Layer

In [76]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

### Siamese Model

In [81]:
def make_siamese_model():
    input_image = Input(name='input_img', shape=(100, 100, 3))
    validation_image = Input(name='validation_img', shape=(100, 100, 3)) 

    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    classifier = Dense(1, activation='sigmoid')(distances)
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [82]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

## Training

### Setting up Loss and Optimizer

In [84]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [87]:
opt = tf.keras.optimizers.Adam(1e-4)

### Checkpoints

In [88]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

### Train Step Function

In [89]:
@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]

        yhat = siamese_model(X, training=True)
        loss = binary_cross_loss(y, yhat)

    print(loss)
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    return loss

### Training Loop

In [93]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print(f'\n Epoch {epoch}/{EPOCHS}')
        progbar = tf.keras.utils.Progbar(len(data))

        for i, batch in enumerate(data):
            train_step(batch)
            progbar.update(i+1)
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

### Training the Model

In [96]:
EPOCHS = 50

In [ ]:
train(train_data, EPOCHS)